In [15]:
from pyspark.context import SparkContext
from pyspark.mllib.regression import LabeledPoint
import numpy as np

In [16]:
sc = SparkContext.getOrCreate()

#sqlContext = SparkSession.builder.master("local[*]").appName("appName")\
#                         .config("spark.sql.warehouse.dir", "./spark-warehouse").getOrCreate()


In [17]:
#Reading the data




dataRDD = sc.textFile("HR_comma_sep.csv")\
         .map(lambda line: line.split(","))\
         .filter(lambda line: len(line)>1)\
         .map(lambda line: [x.encode('UTF8') for x in line])\
         .cache()


print dataRDD.take(5)

[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'left', 'promotion_last_5years', 'sales', 'salary'], ['0.38', '0.53', '2', '157', '3', '0', '1', '0', 'sales', 'low'], ['0.8', '0.86', '5', '262', '6', '0', '1', '0', 'sales', 'medium'], ['0.11', '0.88', '7', '272', '4', '0', '1', '0', 'sales', 'medium'], ['0.72', '0.87', '5', '223', '5', '0', '1', '0', 'sales', 'low']]


In [18]:
header = dataRDD.first()
dataRDD = dataRDD.filter(lambda x: x != header)
all_depts = list(set(dataRDD.map(lambda line: line[-2]).collect()))
all_salaries = list(set(dataRDD.map(lambda line: line[-1]).collect()))

In [19]:
print dataRDD.collect()[:5]

[['0.38', '0.53', '2', '157', '3', '0', '1', '0', 'sales', 'low'], ['0.8', '0.86', '5', '262', '6', '0', '1', '0', 'sales', 'medium'], ['0.11', '0.88', '7', '272', '4', '0', '1', '0', 'sales', 'medium'], ['0.72', '0.87', '5', '223', '5', '0', '1', '0', 'sales', 'low'], ['0.37', '0.52', '2', '159', '3', '0', '1', '0', 'sales', 'low']]


In [20]:
print all_depts
dept_dictionary = {}
for i in all_depts: 
    dept_dictionary[i] = all_depts.index(i)

print all_salaries 

salary_dictionary = {}

for i in all_salaries: 
    salary_dictionary[i] = all_salaries.index(i)
    
print salary_dictionary
print dept_dictionary

['management', 'product_mng', 'hr', 'marketing', 'support', 'sales', 'RandD', 'IT', 'technical', 'accounting']
['high', 'medium', 'low']
{'high': 0, 'medium': 1, 'low': 2}
{'management': 0, 'product_mng': 1, 'hr': 2, 'marketing': 3, 'support': 4, 'sales': 5, 'RandD': 6, 'IT': 7, 'technical': 8, 'accounting': 9}


In [21]:
def parsePoint(line):
    """Converts a list of strings into a LabeledPoint"""
    
    line[-2], line[-1] = dept_dictionary[line[-2]], salary_dictionary[line[-1]]
    
    datapoints = line[1:-4] + line[-3:]
    
    return LabeledPoint(float(line[-4]),[float(i) for i in datapoints])

In [22]:
parsedData = dataRDD.map(parsePoint)
firstPointFeatures = parsedData.first().features
firstPointLabel = parsedData.first().label
print firstPointFeatures, firstPointLabel

[0.53,2.0,157.0,3.0,0.0,0.0,5.0,2.0] 1.0


In [23]:
weights = [.8, .1, .1]
seed = 42
parsedTrainData, parsedValData, parsedTestData = parsedData.randomSplit(weights,seed)
parsedTrainData.cache()
parsedValData.cache()
parsedTestData.cache()
nTrain = parsedTrainData.count()
nVal = parsedValData.count()
nTest = parsedTestData.count()
print nTrain, nVal, nTest, nTrain + nVal + nTest 
print parsedData.count()

12068 1489 1442 14999
14999


In [24]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel

model = LogisticRegressionWithLBFGS.train(parsedTrainData)

labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

model.save(sc, "target/tmp/pythonLogisticRegressionWithLBFGSModel")
sameModel = LogisticRegressionModel.load(sc, "target/tmp/pythonLogisticRegressionWithLBFGSModel")


IllegalArgumentException: u"Error while instantiating 'org.apache.spark.sql.hive.HiveSessionState':"